Este es un archivo de prueba

In [ ]:
print("Hola Mundo")

Hola Mundo


In [2]:
#!pip install sunpy[all]
#!pip install radyn-xtools
#!pip install lightweaver

In [4]:
import os
from os.path import dirname, join as pjoin
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import matplotlib.gridspec as gridspec
import glob
%matplotlib inline

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import time_support
from astropy.io import fits

from datetime import datetime, timedelta

import sunpy.map
from sunpy import timeseries as ts
from sunpy.net import Fido
from sunpy.net import attrs as a
from sunpy.coordinates import frames

import scipy.io as sio
from scipy.io import readsav
from scipy import signal
from scipy import stats
from scipy import integrate
from scipy.optimize import curve_fit
import scipy.constants as const

import pickle
import math

from typing import List

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#folder= "/content/drive/Othercomputers/UofG_Computer/IR/Delay-observations-mid-infrared-channels-in-solar-flare-footpoints"
#folder=r"G:\Other computers\UofG_Computer\IR\Delay-observations-mid-infrared-channels-in-solar-flare-footpoints"
folder=r"C:\Users\migue\OneDrive - University of Glasgow\IR\Delay-observations-mid-infrared-channels-in-solar-flare-footpoints"

#DESCRIPTION OF OBSERVATIONS

In [5]:
# Colourblind-friendly palette from colour universal design (CUD)
CB = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']


# https://jfly.uni-koeln.de/color/
named_colors = {"black": "#000000",
    "orange": "#e69f00",
    "skyblue": "#56b4e9",
    "bluish green": "#009e73",
    "yellow": "#f0e442",
    "blue": "#0072b2",
    "vermilion": "#d55e00",
    "reddish purple": "#cc79a7",
}
cud_palette = list(named_colors.values())


def cud(n: int = len(cud_palette), start: int = 0) -> List[str]:
    remainder = cud_palette[:start]
    palette = cud_palette[start:] + remainder
    return palette[:n]

palette = cud()

In [ ]:
df_10 = pd.read_csv(f"{folder}/intensity_data_10um.csv", parse_dates=['time_10'])
df_05 = pd.read_csv(f"{folder}/intensity_data_05um.csv", parse_dates=['time_05'])

sp_I_10 = df_10['sp_I_10'].values
fA_I_10 = df_10['fA_I_10'].values
fB_I_10 = df_10['fB_I_10'].values
qs_I_10 = df_10['qs_I_10'].values
time_10 = df_10['time_10']

sp_I_05 = df_05['sp_I_05'].values
fA_I_05 = df_05['fA_I_05'].values
fB_I_05 = df_05['fB_I_05'].values
qs_I_05 = df_05['qs_I_05'].values
time_05 = df_05['time_05']

In [ ]:
plt.figure(figsize=(10, 5))
plt.xlabel('Time HH:MM:SS',fontsize=12)
plt.ylabel('Average Intensity ROI',fontsize=12)

plt.plot(time_05, fA_I_05, color=palette[4], label='flare A 5.2um')
plt.plot(time_05, fB_I_05, color=palette[5], label='flare B 5.2um')
plt.plot(time_05, qs_I_05, color=palette[0], label='quiet sun 5.2um')
plt.plot(time_05, sp_I_05, color=palette[1], label='Sunspot penumbra 5.2um')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=2))
plt.xticks(rotation=0)
plt.grid(True)
plt.legend(fontsize=12)#,loc='lower right')
plt.tight_layout()

plt.show()

In [ ]:
y=np.arange(0, len(time_10), 1)*((1.067-1.05)/len(time_10))+1.05 #Linear estimate of the  gradual increase in intensity.

plt.figure(figsize=(10, 5))
plt.xlabel('Time HH:MM:SS',fontsize=12)
plt.ylabel('Average Intensity ROI',fontsize=12)

plt.plot(time_10, fA_I_10, color=palette[2], label='flare A 8.2 µm')
plt.plot(time_10, fB_I_10, color=palette[3], label='flare B 8.2 µm')
plt.plot(time_10, qs_I_10, color=palette[0], label='quiet sun 8.2 µm')
plt.plot(time_10, sp_I_10, color=palette[1], label='Sunspot penumbra 8.2 µm')

plt.plot(time_10, y, 'grey')
plt.annotate('gradual increase', xy=(datetime(2014,9,24,17,52,00), 1.053), xytext=(datetime(2014,9,24,17,52,30), 1.03), arrowprops=dict(facecolor='k', width=0.8), size=10)
plt.gca().add_patch(Rectangle((datetime(2014,9,24,17,55,30), 1), datetime(2014,9,24,17,52,30)-datetime(2014,9,24,17,52,00),0.1, edgecolor = 'grey', facecolor = 'blue', fill=False, lw=3))
plt.annotate('small peaks', xy=(datetime(2014,9,24,17,56,00), 1.03), xytext=(datetime(2014,9,24,17,57,0), 1.029), arrowprops=dict(facecolor='k', width=0.8), size=10)

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=2))
plt.xticks(rotation=0)
plt.grid(True)
plt.legend(fontsize=12,loc='upper right')
plt.tight_layout()

plt.show()

#### GOES data

In [ ]:
file_goes15 = f"{folder}/goes_data.nc"
goes_15 = ts.TimeSeries(file_goes15)

In [ ]:
from datetime import datetime

df = goes_15.to_dataframe()
goes_15 = ts.TimeSeries(df, goes_15.meta, goes_15.units)

goes_flare = goes_15.truncate("2014-09-24 17:44", "2014-09-24 18:01")
time_support()

fig, ax = plt.subplots(figsize=(10, 5))
ax.set_title("2014-09-24 - GOES X-ray fluxes GOES C 7.0 class flare")

ax.plot(goes_flare.time.datetime, goes_flare.quantity("xrsb"), label = "1.0 - 8.0  Å", color="C1")
ax.plot(goes_flare.time.datetime, goes_flare.quantity("xrsa"), label ="0.5 - 4.0  Å", color="C0")

ax.set_ylabel("Wm$^{-2}$")

# Líneas verticales
# Convertir cada Time a datetime compatible con matplotlib
from astropy.time import Time

start = Time('2014-09-24T17:45:00')
peak  = Time('2014-09-24T17:50:00')
end   = Time('2014-09-24T17:52:00')

ax.axvline(start.datetime, color="grey", linestyle="--", linewidth=1, label="Start")
ax.axvline(peak.datetime,  color="red",  linestyle="--", linewidth=1.2, label="Peak")
ax.axvline(end.datetime,   color="blue", linestyle="--", linewidth=1, label="End")

ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=2))
ax.grid(True)

ax.legend(fontsize=12, loc='upper right')
ax.set_xlabel('Time HH:MM:SS', fontsize=12)
ax.set_yscale('log')

plt.show()

In [ ]:
# --- DataFrame ---
df = goes_15.to_dataframe()
goes_15 = ts.TimeSeries(df, goes_15.meta, goes_15.units)

goes_flare = goes_15.truncate("2014-09-24 17:48:50", "2014-09-24 17:49:50")

# === Original Data ===
times = goes_flare.time.datetime
flux_a = goes_flare.quantity("xrsa").value   # 1-8 Å
flux_b = goes_flare.quantity("xrsb").value   # 0.5-4 Å

# === Derivatives ===
# convert datetime to seconds relative to t0
t_seconds = (times - times[0]).astype("timedelta64[s]").astype(float)

dflux_a = np.gradient(flux_a, t_seconds)  # d(flux)/dt
dflux_b = np.gradient(flux_b, t_seconds)
# Gradient in numpy uses: second-order accurate central difference

# === Plot ===
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Panel 1: Flux
ax1.set_title("2014-09-24 GOES X-ray fluxes and derivatives (C7.0 flare)")
ax1.plot(times, flux_b, 'x',label="1.0–8.0 Å", color="C1")
ax1.plot(times, flux_a,'x', label="0.5–4.0 Å", color="C0")
ax1.set_ylabel("Flux (W m$^{-2}$)")
ax1.set_yscale("log")
ax1.legend()
ax1.grid(True)

# Panel 2: Derivatives
ax2.step(times, dflux_b, where='mid', color="C1", label="d/dt (1–8 Å)")
ax2.step(times, dflux_a, where='mid', color="C0", label="d/dt (0.5–4 Å)")

ax2.axhline(0, color="k", linestyle="--", linewidth=0.8)
ax2.set_ylabel("Derivative (W m$^{-2}$ s$^{-1}$)")
ax2.legend()
ax2.grid(True)
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
ax2.xaxis.set_major_locator(mdates.MinuteLocator(interval=2))
ax2.set_xlabel("Time (HH:MM:SS)")

plt.tight_layout()
plt.show()

In [ ]:
#Plots the intesity variation time series between two given times 8.2 um

t1=datetime(2014,9,24,17,47,0) #the initial date of the time series is 2014, 9, 24, 17, 47, 49, 483000
p_t1 = min(enumerate(time_10), key=lambda x: abs(x[1] - t1))[0] #find the position of the closer index given a date

t2=datetime(2014, 9, 24, 18, 1, 7, 483000) #the final date of the time series is 2014, 9, 24, 18, 1, 7, 483000
p_t2 = min(enumerate(time_10), key=lambda x: abs(x[1] - t2))[0] #find the position of the closer index given a date

# # Shaded area times
t1_s = datetime(2014, 9, 24, 17, 48, 50)
t2_s = datetime(2014, 9, 24, 17, 49, 50)

plt.figure(figsize=(10, 5))
plt.xlabel('Time MM:SS', fontsize=12)
plt.ylabel(r'$I-I_0$', fontsize=12)

plt.plot(time_10[p_t1:p_t2], np.subtract(fA_I_10-fA_I_10[0], qs_I_10-qs_I_10[0])[p_t1:p_t2], color=palette[2], label='flare A 8.2 µm')
plt.plot(time_10[p_t1:p_t2], np.subtract(fB_I_10-fB_I_10[0], qs_I_10-qs_I_10[0])[p_t1:p_t2], color=palette[3], label='flare B 8.2 µm')
plt.plot(time_05[p_t1:p_t2], np.subtract(fA_I_05-fA_I_05[0],qs_I_05-qs_I_05[0])[p_t1:p_t2], color=palette[4], label='flare A 5.2 µm')
plt.plot(time_05[p_t1:p_t2], np.subtract(fB_I_05-fB_I_05[0],qs_I_05-qs_I_05[0])[p_t1:p_t2], color=palette[5], label='flare B 5.2 µm')
plt.plot(time_10[p_t1:p_t2], np.subtract(qs_I_10-qs_I_10[0], qs_I_10-qs_I_10[0])[p_t1:p_t2], color=palette[0], label='quiet sun')

# Shaded area
plt.axvspan(t1_s, t2_s, color='gray', alpha=0.3, label='Selected interval')

plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
plt.gca().xaxis.set_major_locator(mdates.MinuteLocator(interval=2))
plt.xticks(rotation=0)

plt.grid(True)
plt.legend(fontsize=12, loc='upper right')
plt.tight_layout()

plt.show()

# ANALYSIS AND RESULTS

## Lag estimation using the Cross-correlation (CCF) and Local CCF for a known signal

In [ ]:
#Using a flare function with a known lag of 0.45s:
#Flare light curve modeled as linear growth followed by a potencial decrese
def flare(x,t0,h,k,a):
    r = []
    for i in range(len(x)):
        if (x[i]<h):
            r_i=(k/(h-t0))*x[i]-k*t0/(h-t0)
            if (r_i>0):
                r.append(r_i)
            else:
                r.append(0)
        else:
            r_i=k*a**((x[i]-h))
            r.append(r_i)
    return r

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

#the analysed signals have arround 1min lenght
lt=61#lenght of time series
st=60/(lt-1) #step size in time
na=0 #Noise Amplitude
ns_1=6#Number of seconds to consider un the fit
ns_2=5
ttl=0.75 #theoretical time lag

noise_1 = np.random.uniform(low=-na, high=na, size=(lt))
noise_2 = np.random.uniform(low=-na, high=na, size=(lt))
t=np.linspace(0,60,lt)
Int_1=(flare(t,10,15,100,0.9)+noise_1)*0.6
Int_2=flare(t, 10+ttl,15+ttl,100,0.9)+noise_2

#Correlation function
mean_Int_1=np.mean(Int_1)
mean_Int_2=np.mean(Int_2)
Norm=np.std(Int_1)*np.std(Int_2)*len(Int_1)

correlation = signal.correlate(Int_1-mean_Int_1, Int_2-mean_Int_2, mode="full")/Norm
lags = signal.correlation_lags(Int_1.size, Int_2.size, mode="full")
lag = lags[np.argmax(correlation)]

x=lags[round(len(lags)/2-ns_1/st):round(len(lags)/2+ns_2/st)]*st
y=correlation[round(len(lags)/2-ns_1/st):round(len(lags)/2+ns_2/st)]

#Fitting
popt,pcov = curve_fit(gaus,x,y)

#Plot
fig = plt.figure(figsize=(15,4))
spec = gridspec.GridSpec(ncols=2, nrows=1, width_ratios=[1, 3], height_ratios=[1])

ax1 = fig.add_subplot(spec[1])

ax1.set_ylabel('Time MM:SS',fontsize=12)
ax1.set_ylabel(r'$I-I_0$',fontsize=12)
ax1.plot(t, Int_1-mean_Int_1, 'b', label='Int_1')
ax1.step(t, Int_1-mean_Int_1, where='mid')
ax1.plot(t, Int_2-mean_Int_2, 'g', label='Int_2')
ax1.step(t, Int_2-mean_Int_2, where='mid')
ax1.grid(True)
ax1.legend(fontsize=12,loc='upper right')

ax2 = fig.add_subplot(spec[0])
ax2.plot(x,y,'x',label='data')
x_ls=(np.linspace(-ns_1, ns_2, num=50))
ax2.plot(x_ls,gaus(x_ls,*popt),'r',label='fit')
ax2.axvline(0, color='r', linestyle='--')
ax2.axvline(popt[1], color='r', linestyle='--')
ax2.legend()
ax2.set_title('Correlation function')
ax2.set_xlabel('Time Lag (s)')
ax2.set_ylabel('Correlation')

plt.show()
print(f"Estimated time lag (max value of the fitting function): {round(popt[1],2)}s")
perr = np.sqrt(np.diag(pcov))
print(f"Standard deviation error {round(perr[1],2)}s")
print(f"Lag {lag*st}s")

In [ ]:
def localCCF(x0,y0,k):
    N= len(x0)

    if k>=0:
        x=x0
        y=y0
    else:
        y=x0
        x=y0

    k=abs(k)
    x_ave=(1/(N-k))*np.sum(x[0:N-k])
    y_ave=(1/(N-k))*np.sum(y[(k+1)-1:N])
    c1=np.sum((1/(N-k))*np.square(x[0:N-k]-x_ave))
    c2=np.sum((1/(N-k))*np.square(y[(k+1)-1:N]-y_ave))
    c=0
    for i in range(N-k):
        c=c+((1/(N-k))*(x[i]-x_ave)*(y[i+k]-y_ave))
    ccf= c/(np.sqrt(c1)*np.sqrt(c2))
    return ccf

def TlocalCCF(x0,y0):
    tccf=[]
    k=len(x0)
    for i in range((-k+3),(k-2)):
        tccf.append(localCCF(x0,y0,i))
    return(tccf)

#Note that the way the correlation function is defined here does not match the way the scipy.correlate function is defined with respect to the order of the variables.

#To obtain comparable results, the function TlocalCCF(x,y) should be compared with signal.correlate(y,x)

In [ ]:
#The analysed signals have arround 1min lenght
lt=61#lenght of time series
st=60/(lt-1) #step size in time
na=0 #Noise Amplitude
ns_1=4.1#Number of seconds to consider un the fit
ns_2=5.5
ttl=0.75 #theoretical time lag

noise_1 = np.random.uniform(low=-na, high=na, size=(lt))
noise_2 = np.random.uniform(low=-na, high=na, size=(lt))
t=np.linspace(0,60,lt)
Int_1=(flare(t,10,15,100,0.9)+noise_1)*0.6
Int_2=flare(t, 10+ttl,15+ttl,100,0.9)+noise_2

#Correlation function
mean_Int_1=np.mean(Int_1)
mean_Int_2=np.mean(Int_2)
Norm=np.std(Int_1)*np.std(Int_2)*len(Int_1)

#We try two different cross correlation functions, local CCF and standard CCF
#Local CCF
correlation_1 = TlocalCCF(Int_1,Int_2)
k=len(Int_1)
lags_1 = np.arange(-k+3, k-2, 1) #Len of 95, the lag 0 is at 47

#Standard CCF (used by Paulo)
correlation_2 = signal.correlate(Int_2-mean_Int_2, Int_1-mean_Int_1, mode="full")/Norm
lags_2 = signal.correlation_lags(Int_2.size, Int_1.size, mode="full")

x1=lags_1[56:62]# 58 is the lag equal to zero
y1=correlation_1[56:62] #

x2=lags_2[58:64]# 60is the lag equal to zero
y2=correlation_2[58:64] #


n = len(x1)                          #the number of data
mean = sum(x1*y1)/n
sigma = sum(y1*(x1-mean)**2)/n

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt1,pcov1 = curve_fit(gaus,x1,y1,p0=[1,mean,sigma])
popt2,pcov2 = curve_fit(gaus,x2,y2,p0=[1,mean,sigma])

plt.plot(x1,y1,'ro:',label='Local CCF')
plt.plot(x2,y2,'bo:',label='Standard CCF')
x_ls=np.linspace(-2, 3.5, num=50)
plt.plot(x_ls,gaus(x_ls,*popt1),'r')
plt.plot(x_ls,gaus(x_ls,*popt2),'b')
#plt.axvline(0, color='g', linestyle='--')
plt.axvline(popt1[1], color='r', linestyle='--')
plt.axvline(popt2[1], color='b', linestyle='--')
plt.legend(loc= 'upper right')
plt.title('Fit for Time Lag')
plt.xlabel('Time Lag (s)')
plt.ylabel('Correlation')
plt.show()

print(f"Estimated time lag (max value of the fitting function) using local CCF: {round(popt1[1], 2) }s")
perr = np.sqrt(np.diag(pcov1))
print(f"Standard deviation error {round(perr[1],3)}s")

print(f"Estimated time lag (max value of the fitting function) using standard CCF: {round(popt2[1], 2)}s")
perr = np.sqrt(np.diag(pcov2))
print(f"Standard deviation error {round(perr[1],3)}s")

## Lag estimation using the Cross-correlation (CCF) and Local CCF for a flare signal in mid infrared

In [ ]:
t1=datetime(2014,9,24,17,49,0) #the initial date of the time series is 2014, 9, 24, 17, 47, 49, 483000
p_t1 = min(enumerate(time_10), key=lambda x: abs(x[1] - t1))[0] #find the position of the closer index given a date

t2=datetime(2014, 9, 24, 17, 49, 50, 0) #the final date of the time series is 2014, 9, 24, 18, 1, 7, 483000
p_t2 = min(enumerate(time_10), key=lambda x: abs(x[1] - t2))[0] #find the position of the closer index given a date

#Intended to subtract a vector, for example if necessary subtract the qs value to obtain the relative variations.
subtract= qs_I_10
fA_Isub=np.subtract(fA_I_10,subtract)[p_t1:p_t2]
fB_Isub=np.subtract(fB_I_10,subtract)[p_t1:p_t2]

mean_fA_Isub=np.average(fA_Isub)
mean_fB_Isub=np.average(fB_Isub)

x=fA_Isub-mean_fA_Isub
y=fB_Isub-mean_fB_Isub
Norm=np.std(x)*np.std(y)*len(x)

#We try two different cross correlation functions, local CCF and standard CCF
#Local CCF
correlation_1 = TlocalCCF(y,x)
k=len(x)
lags_1 = np.arange(-k+3, k-2, 1) #Len of 95, the lag 0 is at 47

#Standard CCF (used by Simoes et al 2024)
correlation_2 = signal.correlate(x, y, mode="full")/Norm
lags_2 = signal.correlation_lags(y.size, x.size, mode="full")


x1=lags_1[44:53]# 47 is the lag equal to zero
y1=correlation_1[44:53] #

x2=lags_2[46:55]# 49 is the lag equal to zero
y2=correlation_2[46:55] #


n = len(x)   #the number of data points
mean = sum(x1*y1)/n
sigma = sum(y1*(x1-mean)**2)/n

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt1,pcov1 = curve_fit(gaus,x1,y1,p0=[1,mean,sigma])
popt2,pcov2 = curve_fit(gaus,x2,y2,p0=[1,mean,sigma])

plt.plot(x1,y1,'ro:',label='Local CCF')
plt.plot(x2,y2,'bo:',label='Standard CCF')
x_ls=np.linspace(-3, 5, num=50)
plt.plot(x_ls,gaus(x_ls,*popt1),'r')
plt.plot(x_ls,gaus(x_ls,*popt2),'b')
#plt.axvline(0, color='g', linestyle='--')
plt.axvline(popt1[1], color='r', linestyle='--')
plt.axvline(popt2[1], color='b', linestyle='--')
plt.legend(loc= 'upper right')
plt.title('Fit for Time Lag')
plt.xlabel('Time Lag (s)')
plt.ylabel('Correlation')
plt.show()

print(f"Estimated time lag (max value of the fitting function) using local CCF: {round(popt1[1], 2) }s")
perr = np.sqrt(np.diag(pcov1))
print(f"Standard deviation error {round(perr[1],3)}s")

print(f"Estimated time lag (max value of the fitting function) using standard CCF: {round(popt2[1], 2)}s")
perr = np.sqrt(np.diag(pcov2))
print(f"Standard deviation error {round(perr[1],3)}s")